In [1]:
# use lod environment
import numpy as np
import matplotlib.pyplot as plt
import shutil
import os
import keras
from skimage.transform import resize
from PIL import UnidentifiedImageError

2025-03-12 16:42:03.510014: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-12 16:42:06.230763: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Sort images into house (having a visible facade) or not house (obstructions)
Taking a pre-trained net on a manually labelled dataset showing houses free of obstruction and with obstruction / only parts of houses -> sort images. As the pretrained net is only working on images with 224x 224 pixels, resize first

# MODEL NOT INCLUDED DUE TO LINCENSE RESTRICTIONS

In [2]:
# important paths
data_dir = "./../../data"
image_dir = data_dir + "/images"
rectified_img_dir = image_dir + "/rectified_images"
resized_img_dir = image_dir + "/rectified_images_resized"
dir_all_houses = image_dir + "/rectified_images_house"
models_dir = data_dir + "/models"

### Resize images

In [3]:
def resize_imgs_partly(old_dir, new_dir, img_height, img_width): 
    '''
    Function resizes all imgs from old_dir and saves into new dir, either all of them or if a part of it is already done. If a new_dir folder already exists, the function skips the ones that were already resized.
    -> can be done in parts and does not need 2-3h in one session

    PARAMS:
    * old_dir (str): path to old directory with images directly in folder
    * new_dir (str): path to new directory in which the resized images should be saved
    * img_height, img_width (ints): new dimensions/size to which images should be resized
    '''    
            
    # make dir for new_dir if not existing yet
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)

    names_new_dir = os.listdir(new_dir)
    names_old_dir = os.listdir(old_dir)

    # delete all imgs in resized dir that are not in old_dir
    for img_name in names_new_dir:
        if img_name not in names_old_dir:
            os.remove(new_dir + "/" + img_name)

    count = 0
    # retrieve each img, resize and save in new dir, except if it already exists in new_dir
    for s, img_name in enumerate(names_old_dir):
        # show progress
        if s % int(len(names_old_dir) / 10) == 0:
            print(s, "/", len(names_old_dir), "done")
        # if same name already in new_dir, do nothing, else resize
        if not img_name in os.listdir(new_dir):
            # if image is readable
            try:
                img = plt.imread(old_dir + "/" + img_name)
                new_img = resize(img, (img_height, img_width))

                plt.imsave(new_dir + "/" + img_name, new_img)  

            # if not readable, skip and count how many
            except UnidentifiedImageError:
                count += 1
                
            except IsADirectoryError:
                count += 1

            except OSError:
                count += 1

    return count

In [ ]:
# runs 306min for 190000 images
count_not_able_to_resize = resize_imgs_partly(old_dir = rectified_img_dir, new_dir = resized_img_dir, img_height = 224, img_width = 224)
print(count_not_able_to_resize, "images could not be resized")

0 / 190949 done
19094 / 190949 done
38188 / 190949 done
57282 / 190949 done
76376 / 190949 done
95470 / 190949 done
114564 / 190949 done
133658 / 190949 done
152752 / 190949 done
171846 / 190949 done
190940 / 190949 done
0 images could not be resized


### Sort into house - not house

In [5]:
class_names = ["house", "not"]
# make directory for house images
if os.path.exists(dir_all_houses):
    shutil.rmtree(dir_all_houses)  ######################################### caution when executing!
    
os.makedirs(dir_all_houses)

In [6]:
# read in model
model = keras.models.load_model(models_dir + '/House_or_not_5epochs.keras')
model.summary()

/home/isabell/anaconda3/envs/lod/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 176 variables whereas the saved optimizer has 14 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenet_1.00_224 (Functional) │ (None, 7, 7, 1024)     │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │       262,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,628,936 (40.55 MB)

 Trainable params: 3,535,682 (13.49 MB)

 Non-trainable params: 21,888 (85.50 KB)

 Optimizer params: 7,071,366 (26.98 MB)

In [ ]:
# runs 55min for 190 000 images
stepsize = 1000
all_imgs = os.listdir(resized_img_dir)
print("nr of images:", len(all_imgs))
# if kernel tends to crash: first time: start range with 0, afterwards set value to next pack that did not finish before a kernel crash
for i in range(0,int(np.ceil(len(all_imgs)/stepsize))):
    print("step", i, "/", int(np.ceil(len(all_imgs)/stepsize)))
    imgs = np.array([plt.imread(resized_img_dir + "/" + img_name) for img_name in all_imgs[stepsize*i:stepsize* (i+1)]])
    names = np.array([img_name for img_name in all_imgs[stepsize*i:stepsize* (i+1)]])
    #print("norming starts")
    # normalize from 0 to 1
    imgs_norm = np.array([(img - np.min(img))/np.max(((np.max(img) - np.min(img)), 1)) for img in imgs])
    #print("actual model starts")
    prediction = model.predict(imgs_norm, verbose = False)
    predictions_max = np.argmax(prediction, axis = 1)
    predictions_classes = [class_names[nr] for nr in predictions_max]
    
    # save all img names for which the prediction is house in house-dir
    [shutil.copy(rectified_img_dir + "/" + img_name, dir_all_houses + "/" + img_name) for class_pred, img_name in zip(predictions_classes, names) if class_pred == "house"]
    print("step completed")
        
    del imgs
    del imgs_norm

print("done")
# ca 106.000 images in house

nr of images: 190949
step 0 / 191


2025-03-12 21:48:23.801211: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 602112000 exceeds 10% of free system memory.
2025-03-12 21:48:24.581464: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2025-03-12 21:48:24.646701: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 104603648 exceeds 10% of free system memory.
2025-03-12 21:48:25.124670: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2025-03-12 21:48:25.192848: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 104603648 exceeds 10% of free system memory.


step completed
step 1 / 191
step completed
step 2 / 191
step completed
step 3 / 191
step completed
step 4 / 191
step completed
step 5 / 191
step completed
step 6 / 191
step completed
step 7 / 191
step completed
step 8 / 191
step completed
step 9 / 191
step completed
step 10 / 191
step completed
step 11 / 191
step completed
step 12 / 191
step completed
step 13 / 191
step completed
step 14 / 191
step completed
step 15 / 191
step completed
step 16 / 191
step completed
step 17 / 191
step completed
step 18 / 191
step completed
step 19 / 191
step completed
step 20 / 191
step completed
step 21 / 191
step completed
step 22 / 191
step completed
step 23 / 191
step completed
step 24 / 191
step completed
step 25 / 191
step completed
step 26 / 191
step completed
step 27 / 191
step completed
step 28 / 191
step completed
step 29 / 191
step completed
step 30 / 191
step completed
step 31 / 191
step completed
step 32 / 191
step completed
step 33 / 191
step completed
step 34 / 191
step completed
step 35 